# Batch Normalization(Theory)


本节作为选读,因为里面涉及到一些术语与方法是在后面才会讲解到.

[《Batch Normalization: Accelerating Deep Network Training by  Reducing Internal Covariate Shift》](https://arxiv.org/pdf/1502.03167.pdf)作为2015年精华型方法普遍用于现代的CV方向的神经网络.

### 1 为什么会有BN的产生

#### 1.1 Internal Covariate Shift

covariate shift的概念:如果ML系统实例集合(X,Y)中的输入值X的分布老是变,这不符合IID假设.

机器学习领域有个很重要的假设:**IID独立同分布假设**,就是假设训练数据和测试数据是满足相同分布的,这是通过训练数据获得的模型能够在测试集获得好的效果的一个基本保障.


也就是说,随着网络深度加深或者在训练过程中,其分布**逐渐发生偏移或者变动**,之所以训练收敛慢,一般是整体分布逐渐往非线性函数的取值区间的上下限两端靠近(对于Sigmoid函数来说,意味着激活输入值ZX+b是大的负值或正值),所以这导致反向传播时低层神经网络的梯度消失,这是训练深层神经网络收敛越来越慢的本质原因.在训练过程中,隐层的输入分布老是变来变去,这就是所谓的“Internal Covariate Shift”,Internal指的是深层网络的隐藏层是发生在网络内部的事情,而不是covariate shift问题,只发生在输入层.

#### 1.2 Whiten

根据以往的经验,如果在图像处理中对输入图像进行白化(Whiten)操作,那么网络的收敛速度会加快,白化后数据满足条件:

(1) 特征之间的相关性降低,这个就相当于PCA

(2) 数据标准化,也就是使得每一维特征均值为0,标准差为1.

以sigmoid举例就是说,数据做了白化处理(这里只做标准化,因为PCA是一个非常耗时的操作),将会处于中间线性阶段而不是分处于两极,这样就能够使得梯度下降的更快,从而避免梯度消失.实际上我们的ReLu,Residual Network也是在解决这个问题.

当然BN也是类似于解决这个问题,即神经网络的每一层的输入都相当于上一层的输出,这样每次都先将数据标准化,那么分布就相同了,但是不仅仅只是标准化那么简单,后面将给出详细解释.

所以:

**BN的基本思想:**

让每个隐层节点的激活输入分布固定下来(即标准化数据),这样就避免了"Internal Covariate Shift"问题了,也可以理解为BN实际上是对每一层的节点做简单化的白化处理(也就是不做PCA).

更加直白的意思就是,既然随着网络深度的增加,分布会出现偏移,那么我们就将其强拉会均值为0,方差为1的数据分布,这样就会使得数据保持同一分布,并且由于将数据标准化会使得数据从原来处于极限区域转向线性区域,使得梯度下降的更快,避免了梯度消失的问题.

那么一句话概括BN的机制就是:

<p style="color:orange">在深度神经网络训练过程中使得每一层神经网络的输入保持相同分布从而避免"Internal Covariate Shift",并且由于数据的标准化会使得梯度下降的更快.</p>

### 2 understanding BN

下面我们来举个例子看看BN的机制:


<img src="../../../picture/42.png" width=500 heigth="500">

假设某个隐藏层的神经元的输入值$Z=WA+b$服从正态分布,正态分布均值是-2,方差是0.5.也就是左边绿色的图,那么BN将其拉回均值为0,方差为1,就相当于图中红色的标准正态分布,这就意味BN将数据整体向右拉2并使得方差更大.那么BN将数据$X$拉回标准正态分布有什么好处呢?

我们先先来看看标准正态分布

<img src="../../../picture/43.png" width=500 heigth="500">

可以看到对于标准正态分布而言,以68%的概率$Z=WA+b$将会落入区间[-1,1],以95%的概率落入区间[-2,2].那么如果我们的非线性激活函数是sigmoid函数

<img src="../../../picture/44.png" width=500 heigth="500">

可以看到数据$Z$会以较大概率落在线性区间,这样就使得梯度下降的更快(因为此部分的导数最大).

到这里也就说明了BN处理后好处

1.数据将处于非线性激活函数的线性阶段,导致梯度下降更快

2.由于每一隐藏层都使用BN,会使得数据处于相同的分布解决了"Internal Covariate Shift".

### 3 some problem

上诉的原理与过程看着是很简单,但是之前为什么没有人使用呢?是因为这里我们忽略了一个问题:

如果是仅仅使用上面的标准化,对网络某一层hidden layer n的输出数据做标准化,然后送入网络下一层hidden layer n+1,这样是会影响到本层网络hidden layer n所学习到的特征的.打个比方,比如我网络中间某一层学习到特征数据本身就分布在sigmoid激活函数的两侧,你强制做标准化,方差也限制在了1,把数据变换成分布于sigmoid函数的中间部分,这样就相当于我这一层网络所学习到的特征分布被破坏了.

还有如果所有的数据都是处于非线性函数的线性部分,那么我们知道如果神经网络的激活函数是线性的,那么实际上该神经网络只是在重复一个线性操作,这样的神经网络是无效的.这意味着神经网络的表达程度降低了,深度也没有意义了.

那么根据paper的作者给出了如下形式来保证找到一个线性与非线性的较平衡点,使得我们的神经网络不太过于线性化,同时保证梯度的更新且可以恢复出原始的某一层所学到的特征的.

对于标准化使用我们的普通标准化:

<img src="../../../picture/45.png" width=300 heigth="300">




对于输出的$y$值从新定义为:

<img src="../../../picture/46.png" width=300 heigth="300">

其中:

### scale: $\gamma^{(k)}=\sqrt{Var[x^{(k)}]+\epsilon}$
### shift:$\beta^{(k)}=E[x^{(k)}]$

需要注意的是:

我们训练过程中采用batch随机梯度下降,上面的$E(x^{(k)})$指的是每一批训练数据神经元$x^{(k)}$的平均值;然后分母就是每一批数据神经元$x^{(k)}$激活度的一个标准差了.也就是说,$x^{(k)}=WU+b$,这里的$x^{(k)}$才是需要normal的,而$U$是上一层的输入.

因此我们引入了这个可学习重构参数γ、β,让我们的网络可以学习恢复出原始网络所要学习的特征分布,同时也使得不太过于线性化.

那么整体流程如paper定义的一样:

<img src="../../../picture/49.png" width=500 heigth="500">

即,如果使用我们熟悉的写法为:

$Z^{[l]}=W^{[l]}A^{[l-1]}+b^{[l]}$

$\mu=\frac{1}{m}\sum_{i=1}^{m}Z^{[l](i)}$

$\sigma^{2}=\frac{1}{m}\sum_{i=1}^{m}(Z^{[l](i)}-\mu)$

$\tilde{Z}^{[l]}=\gamma\cdot\frac{Z^{[l]}-\mu}{\sqrt{\sigma^{2}+\epsilon}}+\beta$

$A^{[l]}=g^{[l]}(\tilde{Z}^{[l]})$



原文并没有将这里的scale和shift做一个详细的说明,也没有说明为什么这样会恢复特征并且让其不要太过于线性化,一个直观的理解为:

通过scale和shift将标准化后的值再做稍稍移动处理,这样就不会使得其完全处于线性阶段,就能够保证网络不是线性的,并且每一个batch计算出来的scale和shift都不一样,这样就相当于每一个batch中scale与shift都将不同,于是等价于整体网络的非线性.当然这只是一个简单的直观理解.

另外由于使用BN之后,实际上$b$会被消除,所以我们没有必要加上$b$,这里加上只是为了一个过渡效果.而且$\beta$也充当了$b$的效果.

### 4 Test

我们知道$BN$在每一轮计算$\mu,\sigma$都是在该轮的batch下,那么训练样本进入的时候,可能没有那么多的样本计算$\mu,\sigma$,即使能够计算出来,那么也有极大的概率是有偏的,甚至我们可以考虑只有一个测试样本的特殊情形下是无法计算$\mu,\sigma$的.

所以,由于模型在测试阶段会出现上述问题,那么paper作者引入如下形式:

<img src="../../../picture/50.png" width=200 heigth="200">

也就是说在我们训练模型的时候,将所有batch中的$\mu_B,\sigma^{2}_B$缓存起来,使用$E_{B}[\mu_{B}]$来估计训练样本的$E[X]$,使用$\frac{m}{m-1}E_{B}[\sigma^{2}_{B}]$来估计训练样本的$Var[x]$.

换句话说,就是使用总体样本去无偏估计测试样本.

最后训练时,形式为:

<img src="../../../picture/51.png" width=300 heigth="300">

其中:

$m:batch\;size$

$\gamma=\sqrt{Var[x]+\epsilon}$

$\beta=E[x]$,

$\epsilon$为极小项

**Ps:**

(1) 当模型处于理想状态,也就是$\epsilon=0$,那么整个等式是$y=x$,实际上$\gamma,\beta$是一个需要学习的参数,也就是说如果在理想情况下,第一次迭代是$y=x$的,但是随着迭代次数的增加,那么更新出来的$\gamma,\beta$是与分母部分不等的,也就可以看为两者是不同的分布.

(2) <span style="color:orange">可以对train阶段每个batch计算的mean/variance采用指数加权平均来得到test阶段mean/variance的估计.</span>

(2.1) 

$running\;mean = running\;mean \cdot momentum + (1- momentum) \cdot mean $

$running\;var = running\;var \cdot momentum + (1- momentum) \cdot var$


### 5. Batch Normalization Backward Propagation 

BN层的反向传播分为三个部分:

**Cell 1**

<img src="../../../picture/52.png" width=300 heigth="300">

这里的$f(x)$是指激活函数,不同的激活函数具有不同的导数,我们这里记为$\frac{\partial f}{\partial y}.$

**Cell 2**

<img src="../../../picture/53.png" width=300 heigth="300">

对于$\gamma:$

${\color{Orchid}{\frac{\partial f}{\partial \gamma}=\frac{\partial f}{\partial y_i}\cdot\frac{\partial y_i}{\partial \gamma}=\sum_{i=1}^{m}\frac{\partial f}{\partial y_i}\cdot\hat{x_{i}}}}$

Note: 这里的$1\to m$是因为我们计算的是batch.那么batch的每一个样本实际上都是对$\gamma$有影响的.

对于$\beta:$

${\color{Orchid}{\frac{\partial f}{\partial \beta}=\frac{\partial f}{\partial y_i}\cdot\frac{\partial y_i}{\partial \beta}=\sum_{i=1}^{m}\frac{\partial f}{\partial y_i}}}$

对于$\hat{x_i}:$

${\color{Orchid}{\frac{\partial f}{\partial \hat{x_i}}=\frac{\partial f}{\partial y_i}\cdot\frac{\partial y_i}{\partial \hat{x_i}}=\frac{\partial f}{\partial y_i}\cdot \gamma}}$

**Cell 3**
<img src="../../../picture/54.png" width=300 heigth="300">

这里的求导需要注意一些地方,

我们需要求导最后的$x$,我们也就需要求导$\mu,\sigma^2$.

在我们对$\mu$求导的时候,实际上同级$\sigma^2$中也包含$\mu$.所以我们也是需要让$\sigma^2$对$\mu$求导.


$\frac{\partial f}{\partial \mu}=\frac{\partial f}{\partial \hat{x_i}}\cdot\frac{\partial \hat{x_i}}{\partial \mu }+{\color{Red} {\frac{\partial f}{\partial \sigma^2}\cdot \frac{\partial \sigma^2}{\partial \mu}}}$

(1) $\frac{\partial \hat{x_i}}{\partial \mu }:$

$\frac{\partial \hat{x_i}}{\partial \mu }=\frac{1}{\sqrt{\sigma^2+\epsilon}}\cdot(-1)$

(2) $\frac{\partial f}{\partial \sigma^2}=\frac{\partial f}{\partial \hat{x}}\cdot\frac{\partial \hat{x}}{\partial \sigma^2}:$

$\frac{\partial \hat{x}}{\partial \sigma^2}=-0.5\sum_{i=1}^{m}(x_i-\mu)\cdot(\sigma^2+\epsilon)^{-1.5}$


${\color{Orchid}{\frac{\partial f}{\partial \sigma^2}=\frac{\partial f}{\partial \hat{x_i}}\cdot-0.5\sum_{i=1}^{m}(x_i-\mu)\cdot(\sigma^2+\epsilon)^{-1.5}}}$

(3) $\frac{\partial \sigma^2}{\partial \mu}:$

$\frac{\partial \sigma^2}{\partial \mu}=\frac{1}{m}\sum_{i=1}^{m}2\cdot(x_i-\mu)\cdot(-1)$

(4) $\frac{\partial f}{\partial \mu}:$

$\frac{\partial f}{\partial \mu}=\left ( \sum_{i=1}^{m}\frac{\partial f}{\partial \hat{x_i}}\cdot\frac{-1}{\sqrt{\sigma^2+\epsilon}} \right ) + \left( \frac{\partial f}{\partial \sigma^2}\cdot\frac{1}{m}\sum_{i=1}^{m}-2(x_i-\mu)\right)$

$\because $

$-2(\frac{1}{m}\sum_{i=1}^{m}x_i - \frac{1}{m}\sum_{i=1}^{m}\mu)=-2(\mu-\frac{m\mu}{m})=0$

$\therefore$

${\color{Orchid} {\frac{\partial f}{\partial \mu}=\left ( \sum_{i=1}^{m}\frac{\partial f}{\partial \hat{x_i}}\cdot\frac{-1}{\sqrt{\sigma^2+\epsilon}} \right )}}$

现在我们可以求$x_i$的导数:

$\frac{\partial f}{\partial x_i}=\frac{\partial f}{\partial \hat{x_i}}\cdot {\color{Red} {\frac{\partial \hat{x_i}}{\partial x_i}}} + \frac{\partial f}{\partial \mu}\cdot{\color{Red} {\frac{\partial \mu}{\partial x_i}}} +\frac{\partial f}{\partial \sigma^2}\cdot{\color{Red} {\frac{\partial \sigma^2}{\partial x_i}}} $

(1) $\frac{\partial \hat{x_i}}{\partial x_i}:$

$\frac{\partial \hat{x_i}}{\partial x_i}=\frac{1}{\sqrt{\sigma^2+\epsilon}}$

(2) $\frac{\partial \mu}{\partial x_i}:$

$\frac{\partial \mu}{\partial x_i}=\frac{1}{m}$

(3) $\frac{\partial \sigma^2}{\partial x_i}:$

$\frac{\partial \sigma^2}{\partial x_i}=\frac{2(x_i-\mu)}{m}$

${\color{Orchid} {\frac{\partial f}{\partial x_i}=\left(\frac{\partial f}{\partial \hat{x_i}}\cdot \frac{1}{\sqrt{\sigma^2+\epsilon}}\right) + \left(\frac{\partial f}{\partial \mu}\cdot \frac{1}{m}\right) + \left(\frac{\partial f}{\partial \sigma^2}\cdot \frac{2(x_i-\mu)}{m} \right)}}$

### 6. Batch Normalization Network

Note: 由于bias会因为标准化而消失,所以在BNN中是不需要计算bias的.

这里以3层隐藏层来直观解释:

<img src="../../../picture/56.png" width=500 heigth="500">

### 7. Advantages of Batch Normalization

Batch Normalization在实际工程中被证明了能够缓解神经网络难以训练的问题,BN具有的有事可以总结为以下三点:

**(1) BN使得网络中每层输入数据的分布相对稳定,加速模型学习速度**

BN通过规范化与线性变换使得每一层网络的输入数据的均值与方差都在一定范围内,使得后一层网络不必不断去适应底层网络中输入的变化,从而实现了网络中层与层之间的解耦,允许每一层进行独立学习,有利于提高整个神经网络的学习速度.

数据的均值和方差在一定范围内,意味着loss会呈现"环形",参考5



**(2) BN使得模型对网络中的参数不那么敏感,简化调参过程,使得网络学习更加稳定**

在神经网络中,我们经常会谨慎地采用一些权重初始化方法(例如Xavier)或者合适的学习率来保证网络稳定训练.

当学习率设置太高时或初始权重太大时,容易出现震荡和不收敛而且还会梯度消失.但是使用BN的网络将不会受到参数数值大小的影响.

因为加入BN以后,即使学习率过大,那么在下一次的迭代中,数据依然会由于normalize而处于线性区域.从而避免了梯度消失和震荡.权重初始化过大也是同样的道理.

所以BN使用较大的学习率可以加速学习的过程,抑制了参数微小变化随着网络层数加深被放大的问题,使得网络对参数大小的适应能力更强,此时我们可以设置较大的学习率而不用过于担心模型差异的风险.

**(3) BN允许网络使用饱和性激活函数(例如sigmoid,tanh等),缓解梯度消失问题**

在不使用BN层的时候,由于网络的深度与复杂性,很容易使得底层网络变化累积到上层网络中,导致模型的训练很容易进入到激活函数的梯度饱和区;通过normalize操作可以让激活函数的输入数据落在梯度非饱和区,缓解梯度消失的问题;另外通过自适应学习 $\gamma$ 与 $\beta$ 又让数据保留更多的原始信息.

**(4) BN具有一定的正则化效果**

在Batch Normalization中,由于我们使用mini-batch的均值与方差作为对整体训练样本均值与方差的估计,尽管每一个batch中的数据都是从总体样本中抽样得到,但不同mini-batch的均值与方差会有所不同,这就为网络的学习过程中增加了随机噪音,与Dropout通过关闭神经元给网络训练带来噪音类似,在一定程度上对模型起到了正则化的效果.

**Ps:**

(1) BN 普遍用于CV而很少用于NLP.

(2) 使用BN之后,训练样本的Loss可能没有BL的Loss小,实际上Loss只是作为一个评判而已,当使用BN后整个数据集就一定会**标准(或近似)化**,那么计算Loss(比如binary crossEntorpy)的$A$(output layer)就可能会比BL的$A$(output layer)大,从而loss也大.

**参考:**

[1.Batch Normalization原理与实战](https://zhuanlan.zhihu.com/p/34879333)

[2.深入理解Batch Normalization批标准化](https://www.cnblogs.com/guoyaohua/p/8724433.html)

[3.Batch Normalization 学习笔记](https://blog.csdn.net/hjimce/article/details/50866313)

[4.Deriving the Gradient for the Backward Pass of Batch Normalization](https://kevinzakka.github.io/2016/09/14/batch_normalization/)

[5.Feature Scaling 和 Batch Norm 笔记](https://shomy.top/2017/12/05/Feature-Scaling-Batch-Norm/)